arrancamos otra vez
carga y normalizacion de datos

In [2]:
import pandas as pd
import sqlite3
import os

## 📁 Rutas
#carpeta = r"C:\UniCABA\Programacion Avanzada para Cs Datos\TP-modulo_2"
#csv_path = os.path.join(carpeta, "ESTADISTICAS CLIMATICAS NORMALES.CSV")
#txt_path = os.path.join(carpeta, "ESTACIONES METEOROLOGICAS.TXT")
#db_path = os.path.join(carpeta, "clima.db")
#

#rutas 

base_dir = os.getcwd()
csv_path = os.path.join(base_dir, "datasets", "ESTADISTICAS CLIMATICAS NORMALES.CSV")
txt_path = os.path.join(base_dir, "datasets", "ESTACIONES METEOROLOGICAS.TXT")
db_path = os.path.join(base_dir, "clima.db")
html_path = os.path.join(base_dir, "dashboard_climatico.html")
mapas_path = os.path.join(base_dir, "mapas")



# 📥 Leer archivos
df_csv = pd.read_csv(csv_path, sep=";", encoding="latin1")
df_txt = pd.read_csv(txt_path, sep=",", encoding="latin1")

# 🧼 Normalizar columnas y textos
df_csv.columns = df_csv.columns.str.strip().str.upper()
df_txt.columns = df_txt.columns.str.strip().str.upper()

def normalizar(s):
    return (str(s).upper().strip()
            .replace("Á", "A").replace("É", "E").replace("Í", "I")
            .replace("Ó", "O").replace("Ú", "U").replace("Ñ", "N"))

df_csv["ESTACIÓN"] = df_csv["ESTACIÓN"].apply(normalizar)
df_txt["NOMBRE"] = df_txt["NOMBRE"].apply(normalizar)


transformacion y union

In [3]:
# 📐 CSV a formato largo
df_largo = df_csv.melt(
    id_vars=["ESTACIÓN", "VALOR MEDIO DE"],
    var_name="MES",
    value_name="VALOR"
)

# 🔗 Unir con coordenadas
df_final = df_largo.merge(
    df_txt[["NOMBRE", "LATITUD", "LONGITUD"]],
    left_on="ESTACIÓN",
    right_on="NOMBRE",
    how="left"
).drop(columns=["NOMBRE"])


guardado en sqlite

In [4]:
# 🗃️ Guardar en base SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS datos_climaticos")
df_final.to_sql("datos_climaticos", conn, index=False)
conn.close()

print("✅ Base 'clima.db' actualizada")


✅ Base 'clima.db' actualizada


verifico que existen datos null y busco la informacion de los que no machearon y se lo agrego con update

In [5]:
# 🛠️ Correcciones manuales de LATITUD y LONGITUD
correcciones = [
    {"ESTACIÓN": "BUENOS AIRES", "LATITUD": -34.61315, "LONGITUD": -58.37723},
    {"ESTACIÓN": "LA QUIACA OBS.", "LATITUD": -22.10448, "LONGITUD": -65.59646},
    {"ESTACIÓN": "MALARGÜE AERO", "LATITUD": -35.47791, "LONGITUD": -69.58521},
    {"ESTACIÓN": "PIGÜE AERO", "LATITUD": -37.60255, "LONGITUD": -62.40836},
    {"ESTACIÓN": "PILAR OBS.", "LATITUD": -34.45867, "LONGITUD": -58.91398},
    {"ESTACIÓN": "SAN MIGUEL", "LATITUD": -34.54335, "LONGITUD": -58.71229},
    {"ESTACIÓN": "VILLA MARIA DEL RIO SECO", "LATITUD": -29.90651, "LONGITUD": -63.72260}
]

# 🔄 Aplicar UPDATE en SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

for item in correcciones:
    cursor.execute("""
        UPDATE datos_climaticos
        SET LATITUD = ?, LONGITUD = ?
        WHERE ESTACIÓN LIKE ?
    """, (item["LATITUD"], item["LONGITUD"], item["ESTACIÓN"]))
    print(f"✅ Coordenadas actualizadas para {item['ESTACIÓN']}")

conn.commit()
conn.close()


✅ Coordenadas actualizadas para BUENOS AIRES
✅ Coordenadas actualizadas para LA QUIACA OBS.
✅ Coordenadas actualizadas para MALARGÜE AERO
✅ Coordenadas actualizadas para PIGÜE AERO
✅ Coordenadas actualizadas para PILAR OBS.
✅ Coordenadas actualizadas para SAN MIGUEL
✅ Coordenadas actualizadas para VILLA MARIA DEL RIO SECO


carga desde bases de datos y kpi's

In [7]:
# 🧼 Normalizar etiquetas para evitar errores de coincidencia
df_final["VALOR MEDIO DE"] = df_final["VALOR MEDIO DE"].str.strip().str.upper()

# 🔢 Convertir VALOR a numérico, ignorando errores
df_final["VALOR LIMPIO"] = pd.to_numeric(df_final["VALOR"], errors="coerce")

# 🎯 Función para calcular promedio por categoría
def calcular_promedio(df, categoria):
    filtro = df["VALOR MEDIO DE"] == categoria.upper()
    valores_validos = df.loc[filtro, "VALOR LIMPIO"].dropna()
    return valores_validos.mean()

# 📊 KPIs
temp_prom = calcular_promedio(df_final, "TEMPERATURA (°C)")
humedad_prom = calcular_promedio(df_final, "HUMEDAD RELATIVA (%)")
viento_prom = calcular_promedio(df_final, "VELOCIDAD DEL VIENTO (KM/H)")

# 📢 Mostrar resultados
print(f"🌡️ Temp. Promedio: {temp_prom:.1f} °C")
print(f"💧 Humedad Prom.: {humedad_prom:.1f} %")
print(f"🌬️ Viento Prom.: {viento_prom:.1f} km/h")




🌡️ Temp. Promedio: 15.0 °C
💧 Humedad Prom.: 68.0 %
🌬️ Viento Prom.: 12.7 km/h


graficos mensuales con drilldown

In [8]:
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# 📋 Dropdown de variables disponibles
variables = df_final["VALOR MEDIO DE"].unique().tolist()
dropdown = widgets.Dropdown(options=variables, description="Variable:")
display(dropdown)

# 📈 Función para graficar variable seleccionada
def graficar_variable(var):
    df_var = df_final[df_final["VALOR MEDIO DE"] == var].copy()

    # 🧼 Convertir a numérico y eliminar nulos
    df_var["VALOR LIMPIO"] = pd.to_numeric(df_var["VALOR"], errors="coerce")
    df_var = df_var.dropna(subset=["VALOR LIMPIO", "MES", "ESTACIÓN"])

    # 🔍 Validación
    if df_var.empty:
        print(f"⚠️ No hay datos válidos para la variable: {var}")
        return

    fig = px.line(df_var,
                  x="MES", y="VALOR LIMPIO", color="ESTACIÓN",
                  title=f"Evolución mensual de {var}",
                  width=800, height=500)
    fig.show()

# 🧠 Activar interactividad (fuera de la función)
widgets.interact(graficar_variable, var=dropdown)



Dropdown(description='Variable:', options=('TEMPERATURA (°C)', 'TEMPERATURA MÁXIMA (°C)', 'TEMPERATURA MÍNIMA …

interactive(children=(Dropdown(description='Variable:', options=('TEMPERATURA (°C)', 'TEMPERATURA MÁXIMA (°C)'…

<function __main__.graficar_variable(var)>

mapa interactivo por mes

In [ ]:
print(df_final["VALOR MEDIO DE"].dropna().unique())



['TEMPERATURA (°C)' 'TEMPERATURA MÁXIMA (°C)' 'TEMPERATURA MÍNIMA (°C)'
 'HUMEDAD RELATIVA (%)' 'VELOCIDAD DEL VIENTO (KM/H)'
 'NUBOSIDAD TOTAL (OCTAVOS)' 'PRECIPITACIÓN (MM)'
 'FRECUENCIA DE DÍAS CON PRECIPITACIÓN SUPERIOR A 0.1 MM']


limpieza y normalización clave para asegurar que el DataFrame esté listo para análisis, visualización y exportación.

In [ ]:
df_final["VALOR MEDIO DE"] = df_final["VALOR MEDIO DE"].str.strip().str.upper()
df_final["VALOR LIMPIO"] = pd.to_numeric(df_final["VALOR"], errors="coerce")
df_final["LATITUD"] = pd.to_numeric(df_final["LATITUD"], errors="coerce")
df_final["LONGITUD"] = pd.to_numeric(df_final["LONGITUD"], errors="coerce")




filtrado específico dentro del DataFrame df_mapa, Extraer únicamente los registros que corresponden a la variable "TEMPERATURA MÁXIMA (°C)" dentro del conjunto de datos ya filtrado por mes (df_mapa)

In [12]:
df_max = df_final[df_final["VALOR MEDIO DE"] == "TEMPERATURA MÁXIMA (°C)"]


generacion mapa

In [13]:
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# 🧼 Normalizar etiquetas y convertir columnas
df_final["VALOR MEDIO DE"] = df_final["VALOR MEDIO DE"].str.strip().str.upper()
df_final["VALOR LIMPIO"] = pd.to_numeric(df_final["VALOR"], errors="coerce")
df_final["LATITUD"] = pd.to_numeric(df_final["LATITUD"], errors="coerce")
df_final["LONGITUD"] = pd.to_numeric(df_final["LONGITUD"], errors="coerce")

# 📋 Dropdown de meses
meses = df_final["MES"].dropna().unique().tolist()
dropdown_mes = widgets.Dropdown(options=["Todos"] + meses, description="Mes:")
display(dropdown_mes)

# 📈 Función para mostrar mapas
def mostrar_mapa(mes):
    df_mapa = df_final.copy() if mes == "Todos" else df_final[df_final["MES"] == mes]

    df_mapa = df_mapa.dropna(subset=["VALOR LIMPIO", "LATITUD", "LONGITUD", "ESTACIÓN"])

    for tipo in ["TEMPERATURA MÁXIMA (°C)", "TEMPERATURA MÍNIMA (°C)"]:
        cantidad = df_mapa[df_mapa["VALOR MEDIO DE"] == tipo]["VALOR LIMPIO"].count()
        print(f"🔎 {tipo} en {mes}: {cantidad} registros válidos")

    for tipo in ["TEMPERATURA MÁXIMA (°C)", "TEMPERATURA MÍNIMA (°C)"]:
        df_temp = df_mapa[df_mapa["VALOR MEDIO DE"] == tipo].copy()
        df_temp["VALOR SIZE"] = df_temp["VALOR LIMPIO"].abs()

        fig = px.scatter_map(df_temp,
                             lat="LATITUD", lon="LONGITUD",
                             color="VALOR LIMPIO", size="VALOR SIZE",
                             hover_name="ESTACIÓN",
                             zoom=4, height=500,
                             title=f"Mapa de temperatura {tipo.split()[1]} - {mes}")
        fig.update_layout(mapbox_style="open-street-map")

        if df_temp.empty:
            fig.add_annotation(
                text="⚠️ No se encontraron registros válidos para esta variable.",
                showarrow=False,
                font=dict(size=16),
                xref="paper", yref="paper",
                x=0.5, y=0.5
            )

        fig.show()

# 🧠 Activar interactividad
widgets.interact(mostrar_mapa, mes=dropdown_mes)


Dropdown(description='Mes:', options=('Todos', 'ENE', 'FEB', 'MAR', 'ABR', 'MAY', 'JUN', 'JUL', 'AGO', 'SEP', …

interactive(children=(Dropdown(description='Mes:', options=('Todos', 'ENE', 'FEB', 'MAR', 'ABR', 'MAY', 'JUN',…

<function __main__.mostrar_mapa(mes)>

exportar como html, este codigo se debe ejecutar desde una consola bash

jupyter nbconvert "modulo 2.ipynb" --to html --output TP_climatico.html

